<a href="https://colab.research.google.com/github/JSJeong-me/GPT-Agent/blob/main/FinanceDaraReader/naver-finance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 파이썬을 이용한 네이버(증권) 주식 정보 크롤링 가이드

## 📌 대상 사이트
- 네이버 증권: [https://finance.naver.com/sise/sise_market_sum.naver](https://finance.naver.com/sise/sise_market_sum.naver)

---

## 📌 간단 실습 설명
1. 여러 HTTP 요청
2. 차단 방지
3. 기본 URL과 파라미터 설정
4. 사용자 입력
5. 데이터 저장을 위한 리스트와 시작 시간 기록
6. 페이지별 데이터 수집 반복문
7. 데이터프레임 생성 및 전처리
8. 데이터 엑셀 파일로 저장 및 서식 적용
9. 최종 결과 출력
10. GUI 만들기

---

## 📌 데이터 요청 분석 방법
1. 네이버 증권으로 이동 / 국내증시 / 시가총액
2. 시가총액(억), PER(배), ROE(%), 외국인비율, 부채총계(억), 매출액증가율 선택 및 적용
3. **F12**를 눌러 개발자 도구 실행
4. 수집할 CSS 선택자 확인

---

## ✅ 필요한 도구 준비하기
---
### 💡 설명
#### requests: 인터넷에서 정보를 가져오는 도구
#### BeautifulSoup: 웹페이지에서 원하는 정보만 골라내는 도구
#### pandas: 데이터를 표 형태로 정리하는 도구
#### time: 시간을 측정하고 대기하는 도구
#### HTTPAdapter, Retry: 인터넷 연결이 불안정할 때 재시도하는 도구

In [1]:
!pip install requests beautifulsoup4 pandas time

ERROR: Could not find a version that satisfies the requirement time (from versions: none)
ERROR: No matching distribution found for time


In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

## ✅ 안정적인 연결 설정하기
---
### 💡 설명
#### 이 부분은 네이버 금융 사이트에 안정적으로 접속하기 위한 설정입니다:
#### 세션 설정: 같은 웹사이트에 여러 번 접속할 때 효율적으로 연결하는 방법
#### 재시도 설정: 연결이 실패하면 최대 5번까지 다시 시도
#### 헤더 설정: 일반 웹브라우저처럼 보이게 하여 차단되지 않도록 함 (마치 가면을 쓰는 것과 같음)

In [3]:
session = requests.Session()
retries = Retry(total=5, backoff_factor=1, status_forcelist=[502, 503, 504])
session.mount('https://', HTTPAdapter(max_retries=retries))

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

## ✅ 사용자 입력 받기
---
### 💡 설명
#### 이 부분은 사용자에게 몇 페이지의 주식 정보를 수집할지 물어보는 부분입니다:
#### 빈 리스트 data는 수집한 정보를 담을 그릇
#### start_time은 작업 시작 시간을 기록하여 나중에 총 소요 시간을 계산


In [4]:
pages = int(input("수집할 페이지의 수를 입력해 주세요세요"))

data = []
start_time = time.time()

수집할 페이지의 수를 입력해 주세요세요3


## ✅ 데이터 수집 과정
---
### 💡 설명
#### url: 네이버 금융 사이트 주소와 원하는 정보 항목을 지정
####    ㄴfieldIds: 수집할 항목들 (시가총액, 부채총계, 매출증가율, 외국인비율, PER, ROE)    
#### response: 웹사이트에서 받아온 응답
#### timeout=10: 10초 안에 응답이 없으면 다시 시도
#### raise_for_status(): 오류가 발생하면 알려줌

In [13]:
for page in range(1, pages+1):
    try:
        url = f"https://finance.naver.com/sise/field_submit.naver?menu=market_sum&returnUrl=http://finance.naver.com/sise/sise_market_sum.naver?page={page}&fieldIds=market_sum&fieldIds=debt_total&fieldIds=sales_increasing_rate&fieldIds=frgn_rate&fieldIds=per&fieldIds=roe"
        # 효율적인 URL 구성 및 요청
        response = session.get(
            url,
            headers=headers,
            timeout=10
        )
        response.raise_for_status()

        # Add the rest of your try block code here,
        # including the BeautifulSoup parsing and data extraction

    except Exception as e:
        print(f"페이지 {page} 처리 중 오류 발생: {str(e)}")
        continue

#### 차이점 비교해보기

In [14]:
"https://finance.naver.com/sise/field_submit.naver?menu=market_sum&returnUrl=http://finance.naver.com/sise/sise_market_sum.naver?page=1&fieldIds=market_sum&fieldIds=debt_total&fieldIds=sales_increasing_rate&fieldIds=frgn_rate&fieldIds=per&fieldIds=roe"
"https://finance.naver.com/sise/field_submit.naver?menu=market_sum&returnUrl=http://finance.naver.com/sise/sise_market_sum.naver?page=2&fieldIds=market_sum&fieldIds=debt_total&fieldIds=sales_increasing_rate&fieldIds=frgn_rate&fieldIds=per&fieldIds=roe"
"https://finance.naver.com/sise/field_submit.naver?menu=market_sum&returnUrl=http://finance.naver.com/sise/sise_market_sum.naver?page=2&fieldIds=market_sum&fieldIds=debt_total&fieldIds=sales_increasing_rate"

'https://finance.naver.com/sise/field_submit.naver?menu=market_sum&returnUrl=http://finance.naver.com/sise/sise_market_sum.naver?page=2&fieldIds=market_sum&fieldIds=debt_total&fieldIds=sales_increasing_rate'

## ✅ 웹페이지에서 정보 추출하기
---
### 💡 설명
#### 이 부분은 웹페이지에서 원하는 정보만 골라내는 과정입니다:
#### BeautifulSoup: 웹페이지의 내용을 분석하는 도구   
#### soup.select: 표(table)에서 주식 정보가 있는 행(tr)들을 선택
#### 각 행에서 필요한 정보를 추출:
#### ㄴname: 종목명 (2번째 열)
#### ㄴmarket_sum: 시가총액 (7번째 열)
#### ㄴdebt_total: 부채총계 (8번째 열)
#### ㄴsales_increasing_rate: 매출증가율 (9번째 열)
#### ㄴfrgn_rate: 외국인비율 (10번째 열)
#### ㄴper: PER (주가수익비율) (11번째 열)
#### ㄴroe: ROE (자기자본이익률) (12번째 열)

In [15]:
soup = BeautifulSoup(response.content, 'html.parser')
trs = soup.select("#contentarea > div.box_type_l >table.type_2 > tbody > tr[onmouseover='mouseOver(this)']")

for tr in trs:
    name = tr.select_one('td:nth-child(2)').text
    market_sum = tr.select_one('td:nth-child(7)').text
    debt_total = tr.select_one('td:nth-child(8)').text
    sales_increasing_rate = tr.select_one('td:nth-child(9)').text
    frgn_rate = tr.select_one('td:nth-child(10)').text
    per = tr.select_one('td:nth-child(11)').text
    roe = tr.select_one('td:nth-child(12)').text

In [19]:
#contentarea > div.box_type_l > table.type_2 > tbody > tr:nth-child(2)
#contentarea > div.box_type_l > table.type_2 > tbody > tr:nth-child(2) > td:nth-child(7)

#contentarea > div.box_type_l > table.type_2 > tbody > tr:nth-child(2)
#contentarea > div.box_type_l >table.type_2 > tbody > tr[onmouseover='mouseOver(this)']
#contentarea > div.box_type_l > table.type_2 > tbody > tr:nth-child(2) > td:nth-child(7)
#contentarea > div.box_type_l > table.type_2 > tbody > tr:nth-child(2) > td:nth-child(8)
#contentarea > div.box_type_l > table.type_2 > tbody > tr:nth-child(2)
#contentarea > div.box_type_l > table.type_2 > tbody > tr:nth-child(2) > td:nth-child(7)

#contentarea > div.box_type_l > table.type_2 > tbody > tr:nth-child(2)
#contentarea > div.box_type_l >table.type_2 > tbody > tr[onmouseover='mouseOver(this)']
#contentarea > div.box_type_l > table.type_2 > tbody > tr:nth-child(2) > td:nth-child(7)
#contentarea > div.box_type_l > table.type_2 > tbody > tr:nth-child(2) > td:nth-child(8)
# There are no syntax errors in this code.

# “#contentarea > div.box_type_l > table.type_2 > tbody > tr[onmouseover='mouseOver(this)']”는 onmouseover 속성의 값이 "mouseOver(this)"인 tr 요소를 선택합니다.

# 페이지 구조상 두 번째 tr 요소에만 onmouseover 속성이 지정되어 있다면,
# 두 선택자는 자연스럽게 같은 요소를 가리키게 됩니다.
# 이렇게 하면, 위치에 기반한 선택과 이벤트나 인터랙션을
# 위해 속성에 기반한 선택을 동시에 활용할 수 있습니다.
# 두 선택자는 자연스럽게 같은 요소를 가리키게 됩니다.
# 이렇게 하면, 위치에 기반한 선택과 이벤트나 인터랙션을
# 위해 속성에 기반한 선택을 동시에 활용할 수 있습니다.

## ✅ 데이터 전처리 및 저장
---
### 💡 설명
#### 이 부분은 수집한 데이터를 정리하는 과정입니다:
#### if market_sum != 'N/A' and ...: 모든 정보가 있는 종목만 선택 ('N/A'는 정보 없음을 의미)   
#### replace(',', ''): 숫자에서 쉼표(,) 제거 (예: '1,000' → '1000') -> 이 작업을 하지 않으면 숫자를 텍스트로 인식해버림.
#### float(): 문자열을 숫자로 변환
#### print(): 수집한 정보를 화면에 출력
#### data.append(): 수집한 정보를 리스트에 추가

In [20]:
if market_sum != 'N/A' and debt_total != 'N/A' and sales_increasing_rate != 'N/A' and frgn_rate != 'N/A' and per != 'N/A' and roe != 'N/A':
    market_sum = float(market_sum.replace(',', ''))
    debt_total = float(debt_total.replace(',', ''))
    sales_increasing_rate = float(sales_increasing_rate.replace(',', ''))
    frgn_rate = float(frgn_rate.replace(',', ''))
    per = float(per.replace(',', ''))
    roe = float(roe.replace(',', ''))
    print(name, market_sum, debt_total, sales_increasing_rate, frgn_rate, per, roe)
    # 엑셀 추가
    data.append([name, market_sum, debt_total, sales_increasing_rate, frgn_rate, per, roe])

롯데쇼핑 22433.0 219694.0 -3.91 8.01 -2.18 -7.75


## ✅ 서버 부하 방지 및 오류 처리
---
### 💡 설명
#### 이 부분은 안정적인 데이터 수집을 위한 설정입니다:
#### time.sleep(1.5): 1.5초 동안 대기 (너무 빠르게 요청하면 차단될 수 있음)
#### except Exception as e: 오류가 발생해도 프로그램이 중단되지 않고 계속 실행
#### continue: 오류가 발생해도 다음 페이지로 넘어감

In [35]:
# time.sleep(1.5)

try:
    time.sleep(1.5)

except Exception as e:
  print(f"페이지 {page} 처리 중 오류 발생: {str(e)}")
  # continue

SyntaxError: expected 'except' or 'finally' block (<ipython-input-35-287caf49189b>, line 8)

## ✅ 데이터프레임 생성 및 엑셀 저장
---
### 💡 설명
#### 이 부분은 수집한 데이터를 엑셀 파일로 저장하는 과정입니다:
#### pd.DataFrame: 수집한 데이터를 표 형태로 변환
#### columns=["종목명", ...]: 각 열의 이름 지정
#### dropna(): 빈 값이 있는 행 제거
#### timestamp: 현재 날짜와 시간을 파일명에 포함 (예: 'stock_data_20240320_153045.xlsx')
#### pd.ExcelWriter: 엑셀 파일 생성
#### df.to_excel: 데이터프레임을 엑셀 파일로 저장
#### number_format: 숫자 형식 설정 (소수점 두 자리까지 표시)

In [37]:
!pip install xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 3.4 MB/s eta 0:00:00


In [38]:
df = pd.DataFrame(data, columns=["종목명", "시가총액", "부채총계", "매출증가율", "외국인비율", "PER", "ROE"]).dropna()

timestamp = time.strftime("%Y%m%d_%H%M%S")
with pd.ExcelWriter(f'stock_data_{timestamp}.xlsx', engine='xlsxwriter') as writer:
    df.to_excel(writer, index=False)
    workbook = writer.book
    worksheet = writer.sheets['Sheet1']
    number_format = workbook.add_format({'num_format': '#,##0.00'})
    worksheet.set_column('B:E', 15, number_format)

## ✅ 작업 완료 메시지
---
### 💡 설명
#### 이 부분은 작업이 완료되었음을 알려주는 메시지입니다:
#### len(df): 수집한 종목 수
#### time.time()-start_time: 작업에 소요된 시간 (초)

In [39]:
print(f"총 {len(df)}개 데이터 수집 완료 | 소요 시간: {time.time()-start_time:.2f}초")

총 1개 데이터 수집 완료 | 소요 시간: 618.94초


## ✅ 최종 코드
---


In [40]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# 세션 설정으로 성능 향상 및 안정성 확보
session = requests.Session()
retries = Retry(total=5, backoff_factor=1, status_forcelist=[502, 503, 504])
session.mount('https://', HTTPAdapter(max_retries=retries))

# 웹 크롤링 차단 방지를 위한 헤더 설정
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# 사용자 입력 받기
pages = int(input("수집할 페이지의 수를 입력해 주세요"))

data = []
start_time = time.time()

#만약 내가 원하는 체크항목이 있다면, f12를 누르고 체크칸에 value 값을 찾아서fieldIds에 넣어준다.
for page in range(1, pages+1):
    try:
        url = f"https://finance.naver.com/sise/field_submit.naver?menu=market_sum&returnUrl=http://finance.naver.com/sise/sise_market_sum.naver?page={page}&fieldIds=market_sum&fieldIds=debt_total&fieldIds=sales_increasing_rate&fieldIds=frgn_rate&fieldIds=per&fieldIds=roe"
        # 효율적인 URL 구성 및 요청
        response = session.get(
            url,
            headers=headers,
            timeout=10
        )
        response.raise_for_status()


        soup = BeautifulSoup(response.content, 'html.parser')
        trs = soup.select("#contentarea > div.box_type_l >table.type_2 > tbody > tr[onmouseover='mouseOver(this)']")

        for tr in trs:
            name = tr.select_one('td:nth-child(2)').text
            market_sum = tr.select_one('td:nth-child(7)').text
            debt_total = tr.select_one('td:nth-child(8)').text
            sales_increasing_rate = tr.select_one('td:nth-child(9)').text
            frgn_rate = tr.select_one('td:nth-child(10)').text
            per = tr.select_one('td:nth-child(11)').text
            roe = tr.select_one('td:nth-child(12)').text

            # 데이터 전처리 - 'N/A' 값이 아닌 경우만 처리
            if market_sum != 'N/A' and debt_total != 'N/A' and sales_increasing_rate != 'N/A' and frgn_rate != 'N/A' and per != 'N/A' and roe != 'N/A':
                market_sum = float(market_sum.replace(',', ''))
                debt_total = float(debt_total.replace(',', ''))
                sales_increasing_rate = float(sales_increasing_rate.replace(',', ''))
                frgn_rate = float(frgn_rate.replace(',', ''))
                per = float(per.replace(',', ''))
                roe = float(roe.replace(',', ''))
                print(name, market_sum, debt_total, sales_increasing_rate, frgn_rate, per, roe)
                # 엑셀 추가
                data.append([name, market_sum, debt_total, sales_increasing_rate, frgn_rate, per, roe])

        # 요청 간격 추가로 서버 부하 방지
        time.sleep(1.5)

    except Exception as e:
        print(f"페이지 {page} 처리 중 오류 발생: {str(e)}")
        continue

# 데이터프레임 처리
df = pd.DataFrame(data, columns=["종목명", "시가총액(억)", "부채총계(억)", "매출증가율", "외국인비율", "PER(배)", "ROE(%)"]).dropna()


# 파일 저장 최적화
timestamp = time.strftime("%Y%m%d_%H%M%S")
with pd.ExcelWriter(f'stock_data_{timestamp}.xlsx', engine='xlsxwriter') as writer:
    df.to_excel(writer, index=False)
    # 엑셀 서식 자동 적용
    workbook = writer.book
    worksheet = writer.sheets['Sheet1']
    number_format = workbook.add_format({'num_format': '#,##0.00'})
    worksheet.set_column('B:E', 15, number_format)

print(f"총 {len(df)}개 데이터 수집 완료 | 소요 시간: {time.time()-start_time:.2f}초")


수집할 페이지의 수를 입력해 주세요1
삼성전자 3421551.0 1123399.0 16.2 49.64 11.2 9.03
SK하이닉스 1583405.0 459395.0 102.02 54.41 6.1 31.06
삼성바이오로직스 735939.0 64316.0 23.08 12.86 57.52 10.45
LG에너지솔루션 675090.0 293402.0 -24.08 4.03 -58.0 -4.93
KB금융 424617.0 6980304.0 9.77 75.49 8.49 8.86
한화에어로스페이스 399653.0 319726.0 42.47 45.06 17.09 53.94
현대차 380849.0 2195225.0 7.73 35.53 4.05 12.43
HD현대중공업 379949.0 136865.0 21.08 10.58 43.32 11.39
셀트리온 356994.0 34752.0 63.45 21.5 71.52 2.46
기아 356315.0 369156.0 7.66 38.47 3.82 19.09
신한지주 297009.0 6809432.0 6.26 58.59 6.85 8.11
NAVER 293901.0 111670.0 11.03 48.44 16.2 7.9
두산에너빌리티 273199.0 146537.0 -7.71 26.25 -268.24 1.52
삼성물산 268223.0 247319.0 0.5 27.07 12.6 6.83
한화오션 235325.0 129805.0 45.46 12.65 33.97 11.52
현대모비스 232488.0 204787.0 -3.41 41.86 5.51 9.35
HD한국조선해양 231782.0 225633.0 19.92 32.48 15.67 11.16
HMM 231146.0 59930.0 39.27 7.79 4.45 15.35
삼성생명 217600.0 2794809.0 9.21 21.71 10.33 6.18
하나금융지주 212119.0 5942706.0 17.1 67.23 5.79 9.11
메리츠금융지주 205036.0 1046482.0 -20.47 15.85 

## 저는 다른 항목을 선택하고 싶어요!

In [41]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# 세션 설정으로 성능 향상 및 안정성 확보
session = requests.Session()
retries = Retry(total=5, backoff_factor=1, status_forcelist=[502, 503, 504])
session.mount('https://', HTTPAdapter(max_retries=retries))

# 웹 크롤링 차단 방지를 위한 헤더 설정
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# 사용자 입력 받기
pages = int(input("수집할 페이지의 수를 입력해 주세요세요"))

data = []
start_time = time.time()

#만약 내가 원하는 체크항목이 있다면, f12를 누르고 체크칸에 value 값을 찾아서fieldIds에 넣어준다.
for page in range(1, pages+1):
    try:
        url = f"https://finance.naver.com/sise/field_submit.naver?menu=market_sum&returnUrl=http://finance.naver.com/sise/sise_market_sum.naver?page=1&fieldIds=quant&fieldIds=ask_buy"
        # 효율적인 URL 구성 및 요청
        response = session.get(
            url,
            headers=headers,
            timeout=10
        )
        response.raise_for_status()

        soup = BeautifulSoup(response.content, 'html.parser')
        trs = soup.select("#contentarea > div.box_type_l >table.type_2 > tbody > tr[onmouseover='mouseOver(this)']")

        for tr in trs:
            name = tr.select_one('td:nth-child(2)').text
            quant = tr.select_one('td:nth-child(7)').text
            ask_buy = tr.select_one('td:nth-child(8)').text


            # 데이터 전처리 - 'N/A' 값이 아닌 경우만 처리
            if quant != 'N/A' and ask_buy != 'N/A':
                quant = float(quant.replace(',', ''))
                ask_buy = float(ask_buy.replace(',', ''))
                print(name, quant, ask_buy)
                # 엑셀 추가
                data.append([name, quant, ask_buy])

        # 요청 간격 추가로 서버 부하 방지
        time.sleep(1.5)

    except Exception as e:
        print(f"페이지 {page} 처리 중 오류 발생: {str(e)}")
        continue

# 데이터프레임 처리
df = pd.DataFrame(data, columns=["종목명", "거래량", "매수호가"]).dropna()


# 파일 저장 최적화
timestamp = time.strftime("%Y%m%d_%H%M%S")
with pd.ExcelWriter(f'stock_data_{timestamp}.xlsx', engine='xlsxwriter') as writer:
    df.to_excel(writer, index=False)
    # 엑셀 서식 자동 적용
    workbook = writer.book
    worksheet = writer.sheets['Sheet1']
    number_format = workbook.add_format({'num_format': '#,##0.00'})
    worksheet.set_column('B:E', 15, number_format)

print(f"총 {len(df)}개 데이터 수집 완료 | 소요 시간: {time.time()-start_time:.2f}초")


수집할 페이지의 수를 입력해 주세요세요1
삼성전자 19390304.0 57700.0
SK하이닉스 4997967.0 217500.0
삼성바이오로직스 78581.0 1033000.0
LG에너지솔루션 214348.0 288000.0
KB금융 2292671.0 107900.0
한화에어로스페이스 213653.0 843000.0
삼성전자우 1686067.0 47450.0
현대차 577325.0 186000.0
HD현대중공업 250762.0 428000.0
셀트리온 468416.0 160400.0
기아 851750.0 89500.0
신한지주 2500648.0 59700.0
NAVER 817069.0 185500.0
두산에너빌리티 11387783.0 42650.0
삼성물산 807743.0 157800.0
한화오션 1957149.0 76700.0
현대모비스 204704.0 249500.0
HD한국조선해양 369838.0 327000.0
HMM 2651013.0 22500.0
삼성생명 694148.0 108500.0
하나금융지주 1734080.0 74500.0
메리츠금융지주 313877.0 113900.0
POSCO홀딩스 331807.0 245000.0
삼성화재 152127.0 425000.0
한국전력 4036843.0 29600.0
카카오 3838475.0 41550.0
크래프톤 163369.0 367000.0
현대로템 1001543.0 156000.0
SK스퀘어 780909.0 124700.0
고려아연 28116.0 767000.0
KT&G 376601.0 124400.0
우리금융지주 4745709.0 20250.0
삼성중공업 9351134.0 16870.0
LG화학 281974.0 198700.0
HD현대일렉트릭 235743.0 381500.0
SK이노베이션 377011.0 90800.0
SK 885037.0 180600.0
기업은행 1742846.0 15800.0
KT 310425.0 49850.0
삼성SDI 503464.0 172400.0
하이브 231183.0 280

## ✅ GUI 만들기
---
#### "(전체 코드 선택 후, Ctrl+k) 이 코드를 꼼꼼하게 읽고, pyside6를 활용하여 GUI를 만들어줘."

In [ ]:
import sys
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from PySide6.QtWidgets import (QApplication, QMainWindow, QWidget, QVBoxLayout,
                              QHBoxLayout, QLabel, QLineEdit, QPushButton,
                              QProgressBar, QTableWidget, QTableWidgetItem,
                              QHeaderView, QMessageBox, QFileDialog)
from PySide6.QtCore import Qt, QThread, Signal

# 웹 크롤링 작업을 위한 스레드 클래스
class CrawlerThread(QThread):
    """
    백그라운드에서 웹 크롤링을 수행하는 스레드 클래스
    GUI가 멈추지 않도록 별도 스레드로 실행
    """
    # 시그널 정의
    progress_signal = Signal(int)  # 진행 상황 업데이트
    data_signal = Signal(list)     # 수집된 데이터 전달
    error_signal = Signal(str)     # 오류 메시지 전달
    finished_signal = Signal(int, float)  # 완료 시그널 (데이터 수, 소요 시간)

    def __init__(self, pages):
        super().__init__()
        self.pages = pages

    def run(self):
        # 세션 설정으로 성능 향상 및 안정성 확보
        session = requests.Session()
        retries = Retry(total=5, backoff_factor=1, status_forcelist=[502, 503, 504])
        session.mount('https://', HTTPAdapter(max_retries=retries))

        # 웹 크롤링 차단 방지를 위한 헤더 설정
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }

        data = []
        start_time = time.time()

        for page in range(1, self.pages+1):
            try:
                url = f"https://finance.naver.com/sise/field_submit.naver?menu=market_sum&returnUrl=http://finance.naver.com/sise/sise_market_sum.naver?page={page}&fieldIds=market_sum&fieldIds=debt_total&fieldIds=sales_increasing_rate&fieldIds=frgn_rate&fieldIds=per&fieldIds=roe"
                # 효율적인 URL 구성 및 요청
                response = session.get(
                    url,
                    headers=headers,
                    timeout=10
                )
                response.raise_for_status()

                soup = BeautifulSoup(response.content, 'html.parser')
                trs = soup.select("#contentarea > div.box_type_l >table.type_2 > tbody > tr[onmouseover='mouseOver(this)']")

                for tr in trs:
                    name = tr.select_one('td:nth-child(2)').text
                    market_sum = tr.select_one('td:nth-child(7)').text
                    debt_total = tr.select_one('td:nth-child(8)').text
                    sales_increasing_rate = tr.select_one('td:nth-child(9)').text
                    frgn_rate = tr.select_one('td:nth-child(10)').text
                    per = tr.select_one('td:nth-child(11)').text
                    roe = tr.select_one('td:nth-child(12)').text

                    # 데이터 전처리 - 'N/A' 값이 아닌 경우만 처리
                    if market_sum != 'N/A' and debt_total != 'N/A' and sales_increasing_rate != 'N/A' and frgn_rate != 'N/A' and per != 'N/A' and roe != 'N/A':
                        market_sum = float(market_sum.replace(',', ''))
                        debt_total = float(debt_total.replace(',', ''))
                        sales_increasing_rate = float(sales_increasing_rate.replace(',', ''))
                        frgn_rate = float(frgn_rate.replace(',', ''))
                        per = float(per.replace(',', ''))
                        roe = float(roe.replace(',', ''))
                        # 데이터 추가
                        data.append([name, market_sum, debt_total, sales_increasing_rate, frgn_rate, per, roe])

                # 진행 상황 업데이트
                progress = int((page / self.pages) * 100)
                self.progress_signal.emit(progress)

                # 요청 간격 추가로 서버 부하 방지
                time.sleep(1.5)

            except Exception as e:
                self.error_signal.emit(f"페이지 {page} 처리 중 오류 발생: {str(e)}")
                continue

        # 수집 완료 후 데이터 전송
        self.data_signal.emit(data)
        self.finished_signal.emit(len(data), time.time() - start_time)


class StockCrawlerApp(QMainWindow):
    """
    주식 데이터 크롤링 및 분석을 위한 메인 GUI 애플리케이션
    """
    def __init__(self):
        super().__init__()
        self.df = None  # 데이터프레임 저장 변수
        self.initUI()

    def initUI(self):
        # 메인 윈도우 설정
        self.setWindowTitle('주식 데이터 수집기')
        self.setGeometry(100, 100, 1000, 600)

        # 중앙 위젯 및 레이아웃 설정
        central_widget = QWidget()
        self.setCentralWidget(central_widget)
        main_layout = QVBoxLayout(central_widget)

        # 입력 영역 레이아웃
        input_layout = QHBoxLayout()

        # 페이지 수 입력
        input_layout.addWidget(QLabel('수집할 페이지 수:'))
        self.page_input = QLineEdit('1')
        self.page_input.setFixedWidth(100)
        input_layout.addWidget(self.page_input)

        # 수집 버튼
        self.crawl_button = QPushButton('데이터 수집 시작')
        self.crawl_button.clicked.connect(self.start_crawling)
        input_layout.addWidget(self.crawl_button)

        # 저장 버튼
        self.save_button = QPushButton('엑셀로 저장')
        self.save_button.clicked.connect(self.save_to_excel)
        self.save_button.setEnabled(False)  # 초기에는 비활성화
        input_layout.addWidget(self.save_button)

        input_layout.addStretch()
        main_layout.addLayout(input_layout)

        # 진행 상황 표시
        progress_layout = QHBoxLayout()
        progress_layout.addWidget(QLabel('진행 상황:'))
        self.progress_bar = QProgressBar()
        progress_layout.addWidget(self.progress_bar)
        main_layout.addLayout(progress_layout)

        # 상태 메시지
        self.status_label = QLabel('준비됨')
        main_layout.addWidget(self.status_label)

        # 데이터 테이블
        self.table = QTableWidget()
        self.table.setColumnCount(7)
        self.table.setHorizontalHeaderLabels(["종목명", "시가총액(억)", "부채총계(억)",
                                             "매출증가율", "외국인비율", "PER(배)", "ROE(%)"])
        self.table.horizontalHeader().setSectionResizeMode(QHeaderView.Stretch)
        main_layout.addWidget(self.table)

    def start_crawling(self):
        """크롤링 작업 시작"""
        try:
            pages = int(self.page_input.text())
            if pages <= 0:
                QMessageBox.warning(self, '입력 오류', '페이지 수는 1 이상이어야 합니다.')
                return

            # UI 상태 업데이트
            self.crawl_button.setEnabled(False)
            self.save_button.setEnabled(False)
            self.progress_bar.setValue(0)
            self.status_label.setText('데이터 수집 중...')
            self.table.setRowCount(0)

            # 크롤링 스레드 시작
            self.crawler_thread = CrawlerThread(pages)
            self.crawler_thread.progress_signal.connect(self.update_progress)
            self.crawler_thread.data_signal.connect(self.display_data)
            self.crawler_thread.error_signal.connect(self.show_error)
            self.crawler_thread.finished_signal.connect(self.crawling_finished)
            self.crawler_thread.start()

        except ValueError:
            QMessageBox.warning(self, '입력 오류', '유효한 페이지 수를 입력하세요.')

    def update_progress(self, value):
        """진행 상황 업데이트"""
        self.progress_bar.setValue(value)

    def display_data(self, data):
        """수집된 데이터 테이블에 표시"""
        # 데이터프레임 생성
        self.df = pd.DataFrame(data, columns=["종목명", "시가총액(억)", "부채총계(억)",
                                             "매출증가율", "외국인비율", "PER(배)", "ROE(%)"])

        # 테이블에 데이터 표시
        self.table.setRowCount(len(data))
        for row_idx, row_data in enumerate(data):
            for col_idx, cell_data in enumerate(row_data):
                item = QTableWidgetItem()
                # 숫자 데이터는 오른쪽 정렬, 문자열은 왼쪽 정렬
                if col_idx == 0:  # 종목명
                    item.setText(str(cell_data))
                    item.setTextAlignment(Qt.AlignLeft | Qt.AlignVCenter)
                else:  # 숫자 데이터
                    item.setText(f"{cell_data:,.2f}")
                    item.setTextAlignment(Qt.AlignRight | Qt.AlignVCenter)
                self.table.setItem(row_idx, col_idx, item)

    def show_error(self, error_msg):
        """오류 메시지 표시"""
        self.status_label.setText(f"오류: {error_msg}")

    def crawling_finished(self, data_count, elapsed_time):
        """크롤링 작업 완료 처리"""
        self.crawl_button.setEnabled(True)
        self.save_button.setEnabled(True)
        self.status_label.setText(f"총 {data_count}개 데이터 수집 완료 | 소요 시간: {elapsed_time:.2f}초")

    def save_to_excel(self):
        """데이터를 엑셀 파일로 저장"""
        if self.df is None or self.df.empty:
            QMessageBox.warning(self, '저장 오류', '저장할 데이터가 없습니다.')
            return

        # 파일 저장 다이얼로그
        timestamp = time.strftime("%Y%m%d_%H%M%S")
        default_filename = f'stock_data_{timestamp}.xlsx'
        file_path, _ = QFileDialog.getSaveFileName(
            self, '엑셀 파일 저장', default_filename, 'Excel Files (*.xlsx)')

        if file_path:
            try:
                # 엑셀 파일 저장
                with pd.ExcelWriter(file_path, engine='xlsxwriter') as writer:
                    self.df.to_excel(writer, index=False)
                    # 엑셀 서식 자동 적용
                    workbook = writer.book
                    worksheet = writer.sheets['Sheet1']
                    number_format = workbook.add_format({'num_format': '#,##0.00'})
                    worksheet.set_column('B:G', 15, number_format)

                QMessageBox.information(self, '저장 완료', f'데이터가 성공적으로 저장되었습니다.\n{file_path}')
            except Exception as e:
                QMessageBox.critical(self, '저장 오류', f'파일 저장 중 오류가 발생했습니다: {str(e)}')


if __name__ == '__main__':
    app = QApplication(sys.argv)
    window = StockCrawlerApp()
    window.show()
    sys.exit(app.exec())


# 시가총액 분석 시 효과적으로 활용할 항목 및 상세 분석법

## 1. 시가총액(억)
- **정의**: 기업의 전체 주식 수에 현재 주가를 곱한 값으로 기업의 전체 시장가치를 나타냄.
- **활용법**:
  - 기업의 규모와 시장 영향력을 평가할 수 있으며, 산업 내 경쟁사와의 상대적 비교를 통해 안정성을 판단함.
  - 규모가 클수록 안정적이지만, 성장률은 다소 낮을 수 있음.

## 2. 외국인비율
- **정의**: 기업 주식을 보유한 외국인 투자자의 비율.
- **활용법**:
  - 외국인 투자자의 관심도와 신뢰도를 파악하여 글로벌 시장에서의 기업 평가를 판단.
  - 외국인 비율 증가 시 주가 상승 가능성을 예측할 수 있으며, 급격한 변동은 리스크 신호일 수 있음.

## 3. PER(배)
- **정의**: 주가수익비율로, 현재 주가가 주당 순이익(EPS)에 비해 몇 배인지를 나타냄.
- **활용법**:
  - 기업이 고평가 또는 저평가되어 있는지를 판단하는 지표.
  - 산업 평균 PER과 비교해 기업의 상대적 가치 평가를 수행할 수 있음.

## 4. ROE(%)
- **정의**: 자기자본이익률로 자기자본 대비 순이익 비율.
- **활용법**:
  - 기업의 경영 효율성과 수익성을 평가하여 투자 수익의 지속가능성을 판단.
  - ROE가 높은 기업은 일반적으로 자본 효율성이 뛰어나며 장기적으로 주가가 안정적으로 상승할 가능성이 높음.

## 5. 부채총계(억)
- **정의**: 기업이 가지고 있는 모든 부채의 총합.
- **활용법**:
  - 기업의 재무 안정성과 부실 가능성을 분석하는 주요 지표.
  - 부채 비율이 높으면 경기 침체 시 기업의 재무 상태가 빠르게 악화될 수 있어 리스크 관리에 필수적.

## 6. 매출액증가율
- **정의**: 전년 대비 매출 증가율.
- **활용법**:
  - 기업의 성장 잠재력과 시장 경쟁력을 직접적으로 반영.
  - 꾸준한 매출 성장률은 장기적으로 기업 가치 상승을 의미하며, 투자자에게 긍정적인 지표.

---

## 추가적인 심층 분석법

### 시가총액 대비 거래대금 비율
- 거래대금이 높으면 유동성이 좋고, 가격 변동성도 큼.
- 단기 매매나 주가 상승 가능성을 평가할 때 유용.

### 자산총계 대비 부채총계 비율(부채비율)
- 부채비율이 낮으면 재무 안정성이 높고, 장기적인 투자에 유리.
- 부채비율이 높으면 재무 위험이 증가하므로 투자 시 유의.

### 당기순이익과 주당순이익(EPS)
- 기업의 실제 수익 창출 능력을 평가.
- 꾸준히 증가하는 기업은 장기적으로 투자 가치가 높음.

---

## 추가 활용할 수 있는 분석 지표 및 방법
- **ROE 및 ROA** 분석으로 자본 효율성을 평가.
- **주당순이익(EPS)** 성장률을 통한 기업의 실질적 수익성 분석.
- 산업별 평균 지표와의 비교 분석을 통해 상대적인 평가.

---

## 심화 분석을 위한 질문
- 외국인 투자자의 순매수 추이를 이용하여 주가 움직임을 예측할 수 있는 분석 방법 탐색.
- 매출액증가율과 PER을 동시에 고려하여 기업의 성장성과 가치 평가를 통합적으로 판단할 수 있는 방법 연구.

